# Week 3 — Part 02: Prompts as API contracts

**Estimated time:** 60–90 minutes

## Learning Objectives

- Treat prompts as specifications (contracts)
- Define clear preconditions and postconditions
- Design strict JSON output contracts
- Create validators that make outputs checkable


## Overview

A strong prompt is not “clever wording”. It’s a **specification**.

If you treat the model like a service, your prompt is the API contract.

We’ll practice writing prompts with:

- explicit **role** and **task**
- explicit **output schema**
- explicit **constraints** (no extra keys, no markdown)
- explicit **refusal conditions** (what to do if info is missing)

And we’ll build a small validator so outputs are checkable.

In [ ]:
import json
from dataclasses import dataclass


@dataclass(frozen=True)
class Extracted:
    person: str | None
    company: str | None


def validate_exact_keys(obj: dict, required_keys: set[str]) -> None:
    extra = set(obj.keys()) - required_keys
    missing = required_keys - set(obj.keys())
    if missing:
        raise ValueError(f"missing keys: {sorted(missing)}")
    if extra:
        raise ValueError(f"extra keys: {sorted(extra)}")


def parse_contract_output(text: str) -> Extracted:
    data = json.loads(text)
    if not isinstance(data, dict):
        raise ValueError("output must be a JSON object")
    validate_exact_keys(data, {"person", "company"})

    person = data.get("person")
    company = data.get("company")

    if person is not None and not isinstance(person, str):
        raise ValueError("person must be string or null")
    if company is not None and not isinstance(company, str):
        raise ValueError("company must be string or null")

    return Extracted(person=person, company=company)


print(parse_contract_output('{"person": "Ada", "company": null}'))

## Exercise: Write a contract prompt

Below is a contract prompt template. Your job is to adjust it for new tasks while keeping it checkable.

Key properties:

- “Return ONLY valid JSON”
- “exactly these keys”
- “Use null if not found”

In [ ]:
def build_extraction_prompt(text: str) -> str:
    # Non-verbatim: same idea, slightly different wording and ordering.
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Output: ONLY JSON with keys person, company.\n"
        "Rules: no markdown, no additional keys, use null when missing.\n\n"
        f"Input text:\n{text}\n"
    )


print(build_extraction_prompt("Ada Lovelace founded nothing."))

In [ ]:
def simulate_model_output(prompt: str) -> str:
    # Simulated outputs for practice (stand-in for real LLM call)
    if "Ada" in prompt:
        return '{"person": "Ada Lovelace", "company": null}'
    return '{"person": null, "company": null}'


raw = simulate_model_output(build_extraction_prompt("Ada Lovelace wrote notes."))
print("raw:", raw)
print("parsed:", parse_contract_output(raw))

In [ ]:
def build_refusal_contract(text: str) -> str:
    # TODO: extend contract to return either:
    # {"ok": true, "person": ..., "company": ...}
    # OR {"ok": false, "error": "..."}
    raise NotImplementedError


print("Implement build_refusal_contract().")

## Self-check

- Does your prompt define exact keys?
- Does it forbid extra text?
- Does it define what to do when info is missing?

## References

- Prompting guide: https://www.promptingguide.ai/
- Anthropic cookbook: https://github.com/anthropics/anthropic-cookbook